# Pandemic Simulation with Object-Oriented Programming

## Table of Content

1. [Representing Social Connections](#1)
2. [Simulate Disease Spread](#2)
3. [Plot graph to visualize the simulation](#3)

### 1. Representing Social Connections <a class="anchor" id="1"></a>

In [1]:
#Define a class named Person to represent the records
class Person:

    #constructor method to create a new Person object
    def __init__(self, first_name, last_name):
        self.first_name=first_name
        self.last_name=last_name
        self.friendlist = []

    #add a new social connection to be stored in this Person object
    def add_friend(self, friend_person):
        self.friendlist.append(friend_person)       #friend_person is a reference to another Person object

    #return a string containing the person's first and last name concatenated together
    def get_name(self):
        self.get_name=self.first_name +' '+ self.last_name
        return self.get_name

    #return a list of Person objects for the social connection that have been added
    def get_friends(self):
        return self.friendlist

### 2. Simulate Disease Spread <a class="anchor" id="2"></a>

### 3. Plot graph to visualize the simulation <a class="anchor" id="3"></a>